In [34]:
import math
import os
import random
import re
import sys
import datetime
import pandas as pd
import numpy as np

def OBV_indicator_calculate_OBV(data):
    ## Step 1:
    '''
    In this step we calculate OBV and its moving average
    '''
    '''
    data has two columns 'Close' and 'Volume', indexed by datetime
    
    Create a dataframe called OBV_data with two columns 'OBV' and 'OBV ema':
        Calculate On-Balance Volume (OBV) using data from the previous step
        Calculate exponential weight moving average of OBV (OBV ema) with center of mass = 5
    
    
    Hint: 
        - data you will use: data['Close'], data['Volume']
        - OBV_data['OBV'][0] = 0
    '''
    OBV_data = pd.DataFrame(index=data.index)
    ##################
    # your code here #
    OBV_data['OBV'] = 0
    obv_ind = data['Close'].diff()
    obv_ind = obv_ind.fillna(0)
    obv_ind = np.where(obv_ind == 0, 0, obv_ind)
    obv_ind = np.where(obv_ind > 0, 1,-1)
    obv_ind = data['Volume'].multiply(obv_ind)
    for i in range(len(OBV_data['OBV'])):
        if i != 0: OBV_data['OBV'][i] = (OBV_data['OBV'][i-1]) + obv_ind[i]
    #OBV_data['OBV'] = OBV_data['OBV'].shift() + obv_ind
    OBV_data['OBV'][0] = 0
    OBV_data['OBV ema'] = OBV_data['OBV'].ewm(com=5).mean()
    return OBV_data

In [35]:
df = pd.read_csv("input000-7.txt",delimiter="\t")

In [62]:
df.head()

,Date,Close,Volume
0,2020-01-02,160.619995,22622100
1,2020-01-03,158.619995,21116200
2,2020-01-06,159.029999,20813700
3,2020-01-07,157.580002,21634100
4,2020-01-08,160.089996,27746500


In [61]:
OBV_indicator_calculate_OBV(df)

,OBV,OBV ema
0,0,0.000000e+00
1,-21116200,-1.151793e+07
2,-302500,-7.081055e+06
3,-21936600,-1.186317e+07
4,5809900,-6.938572e+06
...,...,...
248,619500500,6.000588e+08
249,637434000,6.062880e+08
250,620030800,6.085784e+08
251,599758500,6.071085e+08


In [57]:
def OBV_indicator_strategyNreturn(OBV_data):
    ## Step 2:
    '''
    In this step, we try a simple strategy using OBV indicator.
    The idea is when OBV cross OBV ema from below, we buy. And when OBV cross OBV ema from above, we sell.
    '''
    '''
    Suppose the input OBV_data is the output of the previous step
    
    Generate two pd.Series named Buy_idx and Sell_idx respectively.
    Values in Buy_idx(Sell_idx) should be either True or False, where True indicate a Buy(Sell) signal.
    
    To be specific, 
    
    Buy_idx[t] is True if and only if (OBV_data['OBV'][t-1] < OBV_data['OBV ema'][t-1] and OBV_data['OBV'][t] >= OBV_data['OBV ema'][t])
    
    Similar for Sell_idx
    '''
    '''
    If you print a few lines of Buy_idx out, it should look like
    Date
2020-01-02    False
2020-01-03    False
2020-01-06     True
2020-01-07    False
2020-01-08     True
    '''
    ##################
    # your code here #
    Buy_idx = pd.DataFrame(index=OBV_data.index)
    Sell_idx = pd.DataFrame(index=OBV_data.index)
    Buy_idx['signal'] = 0
    Sell_idx['signal'] = 0
    for i in range(len(Buy_idx)):
        if i!= 0:
            Buy_idx['signal'][i] = OBV_data['OBV'][i-1] < OBV_data['OBV ema'][i-1]
            Sell_idx['signal'][i] = OBV_data['OBV'][i-1] > OBV_data['OBV ema'][i-1]
    Buy_idx['signal'][0] = False
    Sell_idx['signal'][0] = False
    #for i in range(len(Buy_idx)):
    #    if i!= 0:
    #        Buy_idx[i] = OBV_data['OBV'][i-1] < OBV_data['OBV ema'][i-1]
    #        Sell_idx[i] = OBV_data['OBV'][i-1] > OBV_data['OBV ema'][i-1]
    Buy_idx = Buy_idx.copy()
    Sell_idx = Sell_idx.copy()
    ##################
    return Buy_idx, Sell_idx

In [58]:
OBV_indicator_strategyNreturn(OBV_data)

(    signal
 0    False
 1    False
 2     True
 3    False
 4     True
 ..     ...
 248  False
 249  False
 250  False
 251  False
 252   True
 
 [253 rows x 1 columns],
     signal
 0    False
 1    False
 2    False
 3     True
 4    False
 ..     ...
 248   True
 249   True
 250   True
 251   True
 252  False
 
 [253 rows x 1 columns])

In [39]:
OBV_data.index

RangeIndex(start=0, stop=253, step=1)

In [40]:
len(OBV_data)

253